In [1]:
import os
import openai
from dotenv import load_dotenv
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By

In [2]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
driver = webdriver.Chrome('drivers/chromedriver108')

# Global Context

In [88]:
base = 'https://www.amazon.com/'

In [89]:
driver.get(base)

In [90]:
doc = bs(driver.find_element(By.TAG_NAME, 'html').get_attribute('innerHTML'), 'html.parser')

In [91]:
title = str(doc.head.title) + '\n'
metas = '\n'.join(list(map(lambda x: str(x), doc.head.find_all('meta'))))
head_string = title + metas

In [92]:
name_prompt = '''The following text is the url and content of head tag of a website.
url: {0}
head: {1}
The name of website is:'''.format(base, head_string)

category_prompt = '''The following text is the url and content of head tag of a website.
url: {0}
head: {1}
The website category is:'''.format(base, head_string)

In [93]:
name_res = openai.Completion.create(
    model="text-davinci-003",
    prompt=name_prompt,
    temperature=0,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    max_tokens=256
)

name_res

<OpenAIObject text_completion id=cmpl-6fFux5mtett06MTX1imgxAvSO8oi7 at 0x10fc3e400> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " Amazon.com"
    }
  ],
  "created": 1675290655,
  "id": "cmpl-6fFux5mtett06MTX1imgxAvSO8oi7",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 3,
    "prompt_tokens": 656,
    "total_tokens": 659
  }
}

In [94]:
category_res = openai.Completion.create(
    model="text-davinci-003",
    prompt=category_prompt,
    temperature=0,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    max_tokens=256
)

category_res

<OpenAIObject text_completion id=cmpl-6fFuyXKLEhaaD3bPiMg65m9k0l5ay at 0x10fa693b0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " Shopping"
    }
  ],
  "created": 1675290656,
  "id": "cmpl-6fFuyXKLEhaaD3bPiMg65m9k0l5ay",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 1,
    "prompt_tokens": 655,
    "total_tokens": 656
  }
}

In [95]:
global_context = 'The website name is {0}. This website is a {1} website.'.format(
    name_res.choices[0].text.strip(),
    category_res.choices[0].text.strip()
)

global_context

'The website name is Amazon.com. This website is a Shopping website.'

# Page Context

In [105]:
body_doc = bs(driver.find_element(By.TAG_NAME, 'body').get_attribute('innerHTML'), 'html.parser')

In [106]:
headings = []
for h in range(6):
    headings = [*headings, *list(map(lambda x: x.get_text().strip(), body_doc.find_all('h' + str(h + 1))))]

In [107]:
headings

['Checkout\n        \n         (1 item)\n        \n\n\n  Are you sure you want to return to your Shopping Cart?\n\n\n\n\n\nStay in checkout\n\n\n\n\n\n      Return to Cart',
 '1',
 'Shipping address',
 '1',
 'Shipping address',
 'Special delivery options',
 'Verify your age',
 'Tax information',
 'Gift options',
 '2',
 'Payment method',
 '2',
 'Choose a payment method',
 'Add ID',
 '3',
 'Offers',
 '3',
 'Offers',
 'Monthly payments terms',
 '4',
 'Items and shipping',
 '4',
 'Items and shipping',
 'Order Summary',
 'Your available balance',
 'Other payment methods']

In [108]:
local_prompt = '''{0}
We are in page {1} of the website. The headings in this page are: {2}.
The page main functionality is'''.format(
    global_context,
    driver.current_url,
    ', '.join(headings)
)

print(local_prompt)

The website name is Amazon.com. This website is a Shopping website.
We are in page https://www.amazon.com/gp/buy/payselect/handlers/display.html?_from=cheetah of the website. The headings in this page are: Checkout
        
         (1 item)
        


  Are you sure you want to return to your Shopping Cart?





Stay in checkout





      Return to Cart, 1, Shipping address, 1, Shipping address, Special delivery options, Verify your age, Tax information, Gift options, 2, Payment method, 2, Choose a payment method, Add ID, 3, Offers, 3, Offers, Monthly payments terms, 4, Items and shipping, 4, Items and shipping, Order Summary, Your available balance, Other payment methods.
The page main functionality is


In [109]:
local_res = openai.Completion.create(
    model="text-davinci-003",
    prompt=local_prompt,
    temperature=0,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    max_tokens=256
)

print(local_res.choices[0].text)

 to allow customers to complete their checkout process. It provides customers with the option to review their order, enter their shipping address, select a payment method, and apply any offers or discounts. It also provides customers with the option to add an ID for verification purposes and to view their available balance.
